In [36]:
import pinecone
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import PromptTemplate


# Initialization & Setup

In [37]:
# Load environment variables
load_dotenv()

True

In [38]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index = pc.Index("restaurant-info-medium-testing")

In [39]:
embeddings_model = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'),
                                        model="text-embedding-3-small")

In [40]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings_model)

## Running Similarity Search

In [41]:
results = vector_store.similarity_search(
    "sushi in LA",
    k=3,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")
    print()

* Sushi Kaneyoshi in LA is a highly praised sushi restaurant offering an omakase experience that includes dishes like oyster, monkfish liver (ankimo), mackerel handroll, and tamago. Customers generally feel positively about the restaurant, appreciating the fresh fish, authentic Japanese setting, and craftsmanship, although some express concerns about the difficulty of securing reservations and the lack of dessert or additional courses. Key pros include the quality of seafood and the restaurant's authentic vibe, while key cons include the limited seating leading to less intimacy and the potential for disruptive patrons. The price range is not specified. Overall, the restaurant is described as refined and superb, with a touch of Los Angeles flair. [{'address': '250 1st St B1, Los Angeles, CA 90012, USA', 'name': 'Sushi Kaneyoshi', 'place_id': 'ChIJIXKeGv_HwoARtFSSk2sS2AI', 'rating': 4.7}]

* Sushi Gen is a well-regarded sushi restaurant in LA known for its fast service and high-quality, 

# Setting Up Chain

In [42]:
retriever = vector_store.as_retriever()

In [92]:
template = """You are a restaurant search assistant, based off the given context and question I want you to align
the user with the most relevant restaurant given the user's question. List ALL out the restaurants in the following format:
______________________
**Restaurant Name**\n
**Address**\n
**Rating**\n
*Cuisine Type**\n
**Summary and Reviews**\n
_________________________


{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [93]:
llm = OpenAI()

In [94]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [95]:
response = rag_chain.invoke("I want a milkshake")

# Display

In [96]:
from IPython.display import display, Markdown

In [97]:
# Convert the response to Markdown for better formatting
formatted_response = f"### Restaurants\n\n{response}"

# Display the formatted response
display(Markdown(formatted_response))

### Restaurants


______________________
Croft Alley Beverly Hills

9433 Brighton Way, Beverly Hills, CA 90210, USA

Rating: 4.3

Cuisine Type: Coffee, All-day breakfast, Yogurt, Waffles

Summary and Reviews: Croft Alley Beverly Hills is known for its delicious waffles and coffee, including regular lattes and almond milk lattes. However, customers have expressed dissatisfaction with slow service and high prices. The price range is considered moderate, with meals costing around $30 and lattes priced at $7.

______________________
Sunday Cafe

Food court, 400 S Baldwin Ave FC7, Arcadia, CA 91007, USA

Rating: 4.8

Cuisine Type: Aguas Frescas, Banana Splits, Hot Dogs, Steak Fries, Steak Fajitas, Mexican Rice, Siu Maai & Haa Gow

Summary and Reviews: Sunday Cafe, located in the Santa Anita mall food court, is known for its delicious Aguas Frescas in a variety of flavors. Customers also rave about their banana splits, hot dogs, and steak fries. The sentiment is generally positive, highlighting the cafe's delicious food and fun atmosphere.

______________________
THE Blvd Restaurant and Lounge



In [99]:
response = rag_chain.invoke("What is the address of Croft Alley")
print(response)


Croft Alley Beverly Hills
9433 Brighton Way, Beverly Hills, CA 90210, USA
4.3
Regular lattes, almond milk lattes, all-day breakfast options, yogurt, and waffles.

Croft Alley Beverly Hills serves a variety of food items, including regular lattes, almond milk lattes, all-day breakfast options, yogurt, and waffles. Overall, customer sentiment is mixed; while some praise the food quality and coffee, many express dissatisfaction with slow service, high prices, and mandatory tips. Key pros include excellent food and coffee, particularly the waffles, while key cons are slow service, high prices for small portions, and a lack of attentiveness from staff. The price range is considered moderate, with meals costing around $30 and lattes priced at $7. The restaurant has a casual vibe but has received criticism for its service and value.


In [100]:
response = rag_chain.invoke("Where is sushi gen")
print(response)

Sushi Gen

422 E 2nd St, Los Angeles, CA 90012, USA

4.6

Japanese

Sushi Gen is a well-regarded sushi restaurant in LA known for its fast service and high-quality, fresh fish. Customers rave about specific items such as the toro (fatty tuna), fatty salmon, sashimi deluxe, and the $23 lunch special, which is considered a great value. While many patrons appreciate the quality and service, some express disappointment over long wait times, inconsistent experiences with certain menu items like the deluxe sushi set, and the restaurant's outdated decor. The price range is considered expensive. Overall, the sentiment is largely positive, with a few notable criticisms.
